In [ ]:
# Import environment definition
from environment import WirelessEnv
from customcallback import PacketDeliveredCountCallback

In [ ]:
env = WirelessEnv()

In [ ]:
env.observation_space

In [ ]:
env.action_space

In [ ]:
for i in range(env.num_agents):
    act_space = env.get_agent_action_space(i)
    print("agent: ", i, "act_space", act_space)
    print(act_space.sample())

In [ ]:
#Hidden terminal
env.reset()
action_dict = {0:0, 1:2, 2:2, 3:3, 4:1, 5:3}

env.step(action_dict)

In [ ]:
#  0->1 & 5->4 
env.reset()
action_dict = {0:1, 1:1, 2:2, 3:3, 4:2, 5:2}

env.step(action_dict)

In [ ]:
#  2->0 & 5->4 
env.reset()
action_dict = {0:0, 1:1, 2:0, 3:3, 4:2, 5:2}

env.step(action_dict)

In [ ]:
checkpoint_path = "/home/aicon/chethan/repo/pg-aicon/wireless/code/phase_two/wmac_marl/logs/wmac_marl/PPO_WirelessEnv_fa061_00000_0_num_sgd_iter=6_2021-02-03_23-34-28/checkpoint_4167/checkpoint-4167"


In [ ]:
    # Create a single environment and register it
def env_creator(_):
        return WirelessEnv()

single_env = WirelessEnv()
env_name = "WirelessEnv"
#register_env(env_name, env_creator)

# Get environment obs, action spaces and number of agents
obs_space = single_env.observation_space
#act_space = single_env.action_space
num_agents = single_env.num_agents

# Create a policy mapping
def gen_policy(agent_id):
        act_space = single_env.get_agent_action_space(agent_id)
        return (None, obs_space, act_space, {})

policy_graphs = {}
for i in range(num_agents):
        policy_graphs['agent-' + str(i)] = gen_policy(i)
print(policy_graphs)
def policy_mapping_fn(agent_id):
        return 'agent-' + str(agent_id)

    # Define configuration with hyperparam and training details
config={
                "log_level": "ERROR",
                "num_workers": 6,
                "num_cpus_for_driver": 4,
                "num_cpus_per_worker": 2,
                "num_gpus": 0,
                "num_envs_per_worker": 1,
                "no_done_at_end": True,
                "seed":10,
                "gamma": 0.9392979332914239,

#---------------------------------------------------------------------------------------

                "use_critic": True,
                "use_gae": True,
                "lambda": 0.9844457867596674,
                "kl_coeff": 0.2,
                "rollout_fragment_length": 200,
                "train_batch_size": 2048,
                "sgd_minibatch_size": 128,
                "shuffle_sequences": True,
                "num_sgd_iter": 6,
                "lr": 4.304049744289648e-05,
                "lr_schedule": None,
                "vf_share_layers": False,
                "vf_loss_coeff": 1.0,
                "entropy_coeff": 0.05427902707123386,
                "entropy_coeff_schedule": None,
                "clip_param": 0.1,
                "vf_clip_param": 300,
                "grad_clip": None,
                "kl_target": 0.01,
                "batch_mode": "truncate_episodes",
                "observation_filter": "NoFilter",
                "simple_optimizer": False,
                "_fake_gpus": False,
#---------------------------------------------------------------------------------------
                "multiagent": {
                    "policies": policy_graphs,
                    "policy_mapping_fn": policy_mapping_fn,
                    "count_steps_by": "env_steps",
                },
                "env": "WirelessEnv",
                "callbacks": PacketDeliveredCountCallback
}

In [ ]:
from ray.rllib.agents.ppo import PPOTrainer
import ray
import numpy as np

config1={
                "log_level": "ERROR",
                "num_workers": 6,
                "num_cpus_for_driver": 4,
                "num_cpus_per_worker": 2,
                "num_gpus": 0,
                "multiagent": {
                    "policies": policy_graphs,
                    "policy_mapping_fn": policy_mapping_fn,
                    "count_steps_by": "env_steps",
                },
                "env": "WirelessEnv",
}
ray.shutdown()
ray.init()
agent = PPOTrainer(config=config1, env=WirelessEnv)
agent.restore(checkpoint_path)

# instantiate env class
env = WirelessEnv()
i = 0
# run until episode ends
packet_delivered = []
for itr in range(5000):
    episode_reward = 0
    done = {}
    obs = env.reset()
    while (i < 100):
        actions = {}
        for i in range(num_agents):
            actions[i] = agent.compute_action(obs[i], policy_id = 'agent-' + str(i)) 
        obs, reward, done, info = env.step(actions)
        i = i+1
        if done['__all__']:
            packet_delivered.append(env.get_packet_delivered_count())
            if itr % 500 == 0:
                print("pckt delivered mean :", np.mean(packet_delivered))
            break
print("final packt delivered mean :", np.mean(packet_delivered))

In [ ]:
pwd


In [ ]:
if [ -f /home/aicon/chethan/repo/pg-aicon/wireless/code/phase_two/wmac_marl/logs/wmac_marl/PPO_WirelessEnv_fa061_00000_0_num_sgd_iter=6_2021-02-03_23-34-28/checkpoint_4167/checkpoint_4167.tune_metadata ] ; then echo "yes" ; else echo "no" ; fi